In [2]:
import os
import pandas as pd
import numpy
import cv2

In [3]:
def vid2frames(p):
    frames = []
    vid = cv2.VideoCapture(p)
    suc, img = vid.read()
    count = 0
    while suc:
        frames.append(img)
        suc, img = vid.read()
        count += 1
    return frames, count
    

In [ ]:
dir = ""
out_path = ""
# Name of dir where includ, name of path for output
types = os.listdir(dir)
for t in types:
    for labels in os.listdir(os.path.join(dir,t)):
        for l in labels:
             label = l[2:]
             for file in os.listdir(os.path.join(dir,t,label)):
                os.mkdir(os.path.join(out_path,label,file+"frames"))
                frames = []
                vid = cv2.VideoCapture(os.path.join(dir,t,label,file))
                suc, img = vid.read()
                count = 1
                while suc:
                    cv2.imwrite(os.path.join(out_path,label,file+"frames")+str(count)+".jpeg",img)
                    frames.append(img)
                    suc, img = vid.read()
                    count += 1
                

This gives me all the frames of all the videos in the label. and stores these images in a folder